## Documentation

To read more about the search by Query DSL, visit the docs [Query DSL](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl.html), [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-your-data.html) and [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html).

![search_by_query_dsl_docs](../images/search_by_query_dsl_docs.png)

## Connect to ElasticSearch


In [1]:
import { Client } from "npm:@elastic/elasticsearch";
import { load } from "https://deno.land/std/dotenv/mod.ts";

const env = await load({ envPath: "../.env" });

const client = new Client({
  node: env.ELASTICSEARCH_NODE,
  auth: {
    apiKey: env.ELASTICSEARCH_API_KEY,
  },
});

const info = await client.info();
console.log(info);


{
  name: "1e6d9c093984",
  cluster_name: "docker-cluster",
  cluster_uuid: "v3fUyW9OReext6IjPiOCqg",
  version: {
    number: "8.17.4",
    build_flavor: "default",
    build_type: "docker",
    build_hash: "c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1",
    build_date: "2025-03-20T15:39:59.811110136Z",
    build_snapshot: false,
    lucene_version: "9.12.0",
    minimum_wire_compatibility_version: "7.17.0",
    minimum_index_compatibility_version: "7.0.0"
  },
  tagline: "You Know, for Search"
}


## Create index


In [2]:
await client.indices.delete({ index: "my_index", ignore_unavailable: true });
await client.indices.create({ index: "my_index" });


{ acknowledged: true, shards_acknowledged: true, index: "my_index" }

Let's index the documents.

In [3]:
import data from "../data/dummy_data.json" with { type: "json" };

async function insertDocument(index: string, document: any) {
  return await client.index({
    index,
    document,
  });
}

data.forEach(async (document: any) => {
  await insertDocument("my_index", document);
});


## Searching

### 1. Leaf clauses

#### 1.1. term query

You can use the term query to find documents based on a precise value such as a `price`, a `product ID`, or a `username`.


Avoid using the `term` query for `text` fields.

To search `text` field values, use the `match` query instead.

In [4]:
await client.search({
  index: "my_index",
  query: {
    term: {
      createdAt: "2025-03-01",
    },
  },
});


{
  took: 1,
  timed_out: false,
  _shards: { total: 1, successful: 1, skipped: 0, failed: 0 },
  hits: {
    total: { value: 1, relation: "eq" },
    max_score: 1,
    hits: [
      {
        _index: "my_index",
        _id: "7Wx5AZYBTEjA208JO_-3",
        _score: 1,
        _source: {
          title: "Title 1",
          text: "This is the first sample document text.",
          createdAt: "2025-03-01"
        }
      }
    ]
  }
}

#### 1.2. match query

The `match` query is the standard query for performing a `full-text search`, including options for fuzzy matching.

In [5]:
await client.search({
  index: "my_index",
  query: {
    match: {
      text: "third",
    },
  },
});


{
  took: 3,
  timed_out: false,
  _shards: { total: 1, successful: 1, skipped: 0, failed: 0 },
  hits: {
    total: { value: 1, relation: "eq" },
    max_score: 0.94566,
    hits: [
      {
        _index: "my_index",
        _id: "72x5AZYBTEjA208JO__Q",
        _score: 0.94566,
        _source: {
          title: "Title 3",
          text: "The content of the third document goes here.",
          createdAt: "2025-03-03"
        }
      }
    ]
  }
}

#### 1.3. range query

Returns documents that contain terms within a provided range.

In [6]:
await client.search({
  index: "my_index",
  query: {
    range: {
      createdAt: {
        gte: "2025-03-01",
        lte: "2025-03-10",
        format: "yyyy-MM-dd",
      },
    },
  },
});


{
  took: 1,
  timed_out: false,
  _shards: { total: 1, successful: 1, skipped: 0, failed: 0 },
  hits: {
    total: { value: 3, relation: "eq" },
    max_score: 1,
    hits: [
      {
        _index: "my_index",
        _id: "7Wx5AZYBTEjA208JO_-3",
        _score: 1,
        _source: {
          title: "Title 1",
          text: "This is the first sample document text.",
          createdAt: "2025-03-01"
        }
      },
      {
        _index: "my_index",
        _id: "7mx5AZYBTEjA208JO_-3",
        _score: 1,
        _source: {
          title: "Title 2",
          text: "Here is another example of a document.",
          createdAt: "2025-03-02"
        }
      },
      {
        _index: "my_index",
        _id: "72x5AZYBTEjA208JO__Q",
        _score: 1,
        _source: {
          title: "Title 3",
          text: "The content of the third document goes here.",
          createdAt: "2025-03-03"
        }
      }
    ]
  }
}

### 2. Compound clauses

Let's search for documents that meet the following criteria:
- Created on `2025-03-02`
- Have the word `document` in the text field.

In [8]:
await client.search({
  index: "my_index",
  query: {
    bool: {
      must: [
        {
          term: {
            createdAt: "2025-03-02",
          },
        },
        {
          match: {
            text: "document",
          },
        },
      ],
    },
  },
});


{
  took: 1,
  timed_out: false,
  _shards: { total: 1, successful: 1, skipped: 0, failed: 0 },
  hits: {
    total: { value: 1, relation: "eq" },
    max_score: 1.1360614,
    hits: [
      {
        _index: "my_index",
        _id: "7mx5AZYBTEjA208JO_-3",
        _score: 1.1360614,
        _source: {
          title: "Title 2",
          text: "Here is another example of a document.",
          createdAt: "2025-03-02"
        }
      }
    ]
  }
}